In [8]:
from django.conf import settings
import os
import tarfile
import uuid
import shutil
from tardis.tardis_portal.models import StorageBox, DataFileObject, DataFile
from tardis.tardis_portal.models.dataset import Dataset
from tardis.tardis_portal.storage.tararchivefs import file_archive_paths, ensure_dir
from tardis.tardis_portal.storage.tararchivefs import TarArchiveFileSystemStorageUtil
#mkdir -p functionality
def ensure_dir(f):
    d = os.path.dirname(f)
    if not os.path.exists(d):
        os.makedirs(d)

In [2]:
#TODOsteve report bug with verified (unverified if one storage box doesn't work which has weird side effects like image loading and downloading)

In [2]:
boxes = StorageBox.objects.all()

In [3]:
boxes

[<StorageBox: local box at /home/ubuntu/mytardis/var/store>]

In [5]:
boxes[0]

<StorageBox: local box at /home/ubuntu/mytardis/var/store>

In [12]:
default_box = boxes[0]

In [7]:
# copy to tararchive box (won't do this in future, preferring cache)
# default_box.copy_files(dest_box=boxes[1])

In [11]:
# copy from tararchive (cache) to default
boxes[1].copy_files(dest_box=default_box)

<type 'type'>


In [13]:
# df = dataset.datafile_set.all()[0]
# dfo = df.file_objects.first()
# dfo._get_identifier()

In [14]:
# TODO: make <dataset_id>.tar in cache box

In [9]:
# task that goes through and creates tars where applicable
# boxes for dataset
# TODOsteve change 'tar_path' to 'location' option name
# THIS CELL ADDS A TAR ARCHIVE AND DFOS eg. tape mirror

# todo settings
tararchive_class = 'tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage'

default_box = StorageBox.objects.filter(django_storage_class='tardis.tardis_portal.storage.MyTardisLocalFileSystemStorage').first()
# inferred cache path from tar box
dataset_default_path = default_box.options.filter(key='location').first().value
print 'default location: ' + dataset_default_path

boxes = StorageBox.objects.filter(django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage')

for tararchive_box in boxes:
    
    dataset_id = int(tararchive_box.options.filter(key='dataset_id').first().value)
    print 'dataset id: ' + str(dataset_id)
    
    tararchive_util = TarArchiveFileSystemStorageUtil(dataset_id)
    
    dataset_tar, existing_cache_tar = tararchive_util.archive_paths()
    
    # dfos for dataset with default boxes but nothing else
    dfs = DataFile.objects.filter(dataset__id=dataset_id, \
        file_objects__storage_box__django_storage_class='tardis.tardis_portal.storage.MyTardisLocalFileSystemStorage')

    no_disk = False
    no_disk = (dfs.count() == 0)
    if no_disk:
        print 'No disk currently for this dataset so nothing to tar: ' + str(dataset_id)
    else:
        # corresponding tar box
        tararchive_box_ds = StorageBox.objects.filter(options__key='dataset_id',
            options__value=dataset_id, django_storage_class=tararchive_class)

        # remove all old associated dfos from box
        for box in tararchive_box_ds:
            for dfo in box.file_objects.all():
                print dfo
                print file_archive_paths(dfo.file_object.name)
                dfo.delete()

        print 'tararchive box for dataset: ' + str(tararchive_box_ds)

        dataset = Dataset.objects.filter(id=dataset_id).first()
        print 'dataset: ' + str(dataset)

        # tar exists or not
        exists = os.path.isfile(dataset_tar)
        print 'tar already exists: ' + str(exists)

        unique_filename = uuid.uuid4()
        
        dataset_cache_path = os.path.dirname(existing_cache_tar)
        tmp_cache_tar = dataset_cache_path + '/' + str(unique_filename) + '.tar'
        print tmp_cache_tar


        # if tar archive doesn't exist
        # TODOsteve modified times and update/replace tar
        if 1==1:
            # create tar in cache
            with tarfile.open(tmp_cache_tar, "w") as tar:
                for df in dataset.datafile_set.all():

                    dfo = df.file_objects.first()
                    file_path = dfo._storage.path(dfo.uri)

                    info = tarfile.TarInfo(name=dfo.uri)
                    info.size=os.path.getsize(file_path)

                    tar.addfile(info,
                            file(file_path))

            # copy file to tar archive and rename to dataset_id.tar
            existing_cache_tar

            print 'renaming!! ' + str(tmp_cache_tar) \
                + ' to ' + str(existing_cache_tar)
            os.rename(tmp_cache_tar, existing_cache_tar)

            print 'copying temp tar file ' + tmp_cache_tar + ' to ' + os.path.dirname(dataset_tar)
            shutil.copy2(existing_cache_tar, dataset_tar)

            # remove tar from cache
            #os.remove(tmp_cache_tar)

            for df in dataset.datafile_set.all():
                dfo = df.file_objects.first()           
                print '\t' + dfo._storage.path(dfo.uri)

                # create dfo
                # TODOsteve create storage box in process
                copy = DataFileObject(
                    datafile=df,
                    storage_box=tararchive_box,
                    uri='%s.tar/%s' % (dataset_id, dfo.uri))
                copy.save()

                print 'creating dfo copy ' + str(df) + ' in ' + str(tararchive_box)

        # TODOsteve finally clean up on fail/success
        # TODO cover for do nothing because already ONLY on tape

default location: /home/ubuntu/mytardis/var/store
dataset id: 1
tararchive box for dataset: [<StorageBox: tararchive>]
dataset: Test Dataset
tar already exists: False
/home/ubuntu/mytardis/var/tararchive_cache/8aee67ea-aa53-4fb2-ab80-e8f59f9ee4bf.tar
renaming!! /home/ubuntu/mytardis/var/tararchive_cache/8aee67ea-aa53-4fb2-ab80-e8f59f9ee4bf.tar to /home/ubuntu/mytardis/var/tararchive_cache/1.tar
copying temp tar file /home/ubuntu/mytardis/var/tararchive_cache/8aee67ea-aa53-4fb2-ab80-e8f59f9ee4bf.tar to /home/ubuntu/mytardis/var/tararchive
	/home/ubuntu/mytardis/var/store/Test Dataset-1/markcorrigan1.jpg
creating dfo copy dfbb59b5eef3152843c55e5e7866ee31 markcorrigan1.jpg # image/jpeg in tararchive
	/home/ubuntu/mytardis/var/store/Test Dataset-1/markcorrigan2.png
creating dfo copy 9dbfe2d17b0788585ec8e054cedc6705 markcorrigan2.png # image/png in tararchive


In [11]:
def is_tape_only(dataset_id):
    tape_only = True
    for df in dfs:
        if df.file_objects.count() > 1:
            tape_only = False
    
    return tape_only

dataset_id = str(1)
tararchive_util = TarArchiveFileSystemStorageUtil(dataset_id)
# TODOsteve string check for retrieve function
# TODOsteve supply dataset ID to bring back to disk (tape only)
# retrieval from tape stuff (the other side of the fence)
# TODOsteve we can't retreive from tape if dataset data is newer..
# this shouldn't happen as we should lock datasets on tape from writing?
# also need a modified time on datasets to clean them

# TODOsteve need a more robust way to get the default box (there must be a function)
default_box = StorageBox.objects.filter(django_storage_class='tardis.tardis_portal.storage.MyTardisLocalFileSystemStorage').first()
# inferred cache path from tar box
dataset_default_path = default_box.options.filter(key='location').first().value
print 'default location: ' + dataset_default_path

boxes = StorageBox.objects.filter(options__key='dataset_id', options__value=dataset_id,
            django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage')

for tararchive_box in boxes:
    
    # dfos for dataset with tararchive boxes but nothing else
    dfs = DataFile.objects.filter(dataset__id=dataset_id, \
        file_objects__storage_box__django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage')

    tape_only = is_tape_only(dataset_id)

    print 'Tape only dataset: ' + str(tape_only)
    
    if tape_only:

        # removed stuff about comparing tar to cache assuming 1 way sync
        print 'getting data from tape as its different from archived'
        tararchive_util.retrieve_tar_from_tape() 
        print 'copying files back to default box'
        tararchive_box.copy_files(dest_box=default_box)

default location: /home/ubuntu/mytardis/var/store
Tape only dataset: False


In [7]:
# tararchive removal
#dfos = DataFileObject.objects.filter(storage_box=tararchive_box,
#                                     storage_box__options__key='dataset_id',
#                                     storage_box__options__key='value',)

dataset_id = 1
# for when we only want stuff on disk
# remove all old associated dfos from box
tararchive_boxes = StorageBox.objects.filter(options__key='dataset_id', options__value=dataset_id, django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage')

# for box in tararchive_boxes:
#     for dfo in box.file_objects.all():
#         print dfo
#         dfo.delete()


# remove all old associated dfos for dataset from box
# for when we only want stuff on tape
default_boxes = StorageBox.objects.filter(django_storage_class='tardis.tardis_portal.storage.MyTardisLocalFileSystemStorage')

# for box in default_boxes:
#     print box
#     for dfo in box.file_objects.all():
#         print dfo
#         print 'removing dfo from box: ' + dfo.datafile.filename
#         dfo.delete()


#
# other stuff
# for box in tararchive_boxes:
#     print box

# for dfo in dfos:
#     print dfo
    #dfo.delete()

In [72]:
# ALL FILE REMOVAL CARE
# dfos = DataFileObject.objects.all()

# for dfo in dfos:
#    print dfo
#    dfo.delete()
    
#FILE LINKS TOO
# dfs = DataFile.objects.all()
# for df in dfs:
#    print df
#    df.delete()

In [111]:
import tarfile,os

os.chdir("/home/ubuntu/mytardis/var/tararchive")
tar = tarfile.open("1.tar")
#for member in tar.getmembers():
#    print member
#     f=tar.extractfile(member)
#     content=f.read()
#     print "%s has %d newlines" %(member, content.count("\n"))
#     print "%s has %d spaces" % (member,content.count(" "))
#     print "%s has %d characters" % (member, len(content))
#     sys.exit()

#filename = 'A dataset-1/Photo 4-10-12 2 15 00 PM.png'
filename = 'A dataset-1/SekZx.jpg'

write_path = '/home/ubuntu/mytardis/var/tararchive_cache/' + filename

member = tar.getmember(filename)

f=tar.extractfile(member)

ensure_dir(write_path)

with open(write_path, 'w+') as extracted_file:
    extracted_file.write(f.read())

tar.close()

In [24]:
# dfos for dataset with tararchive boxes but nothing else
# for when we want to see if stuff is only on tape
dfs = dfs = DataFile.objects.filter(dataset__id=dataset_id, \
    file_objects__storage_box__django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage')

tape_only = True
for df in dfs:
    if df.file_objects.count() > 1:
        tape_only = False

print 'Tape only dataset: ' + str(tape_only)

Tape only dataset: True


In [17]:


class yaytest():
    def __init__(self):
        import sys
        print sys.modules[self.__class__.__module__]
        print self.__class__

y = yaytest()

<module '__main__' (built-in)>
__main__.yaytest


In [16]:
# parse name from dfo

filename = '1.tar/Test Dataset-1/markcorrigan2.png'

def file_archive_paths(filename):
    '''
    returns ('1.tar', 'Test Dataset-1/file.png')
    from filename '1.tar/Test Dataset-1/file.png'
    '''
    tar_name = filename.split('/')[0]
    file_path = filename[len(tar_name) + 1:]
    return tar_name, file_path

print file_archive_paths(filename)


('1.tar', 'Test Dataset-1/markcorrigan2.png')
